In [3]:
inputPath='E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Input'
outputPath='E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Output'
splitfolders.ratio(inputPath, output=outputPath, seed=1337, ratio=(0.8,0.0,0.2))

In [43]:
import cv2
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout 
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from matplotlib import pyplot as plt
import splitfolders
import pandas as pd
from keras.utils import np_utils
import numpy as np
import tensorflow as tf

In [44]:
X_test = []
Y_test = []
testPath = 'E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Output\\test'
for folder in os.listdir(testPath):
    for image in os.listdir(os.path.join(testPath, folder)):
        X_test.append(cv2.imread(os.path.join(testPath, os.path.join(folder,image))))
        Y_test.append(folder)

In [45]:
X_train = []
Y_train = []
trainPath = 'E:\CSLearning\AI and ML\Projects\Satellite Image Classification\Pictures\Output\\train'
for folder in os.listdir(trainPath):
    for image in os.listdir(os.path.join(trainPath, folder)):
        X_train.append(cv2.imread(os.path.join(trainPath, os.path.join(folder,image))))
        Y_train.append(folder)

In [46]:
Y_train = np.asarray(Y_train)
X_test = np.asarray(X_test)
X_train = np.asarray(X_train)
Y_test = np.asarray(Y_test)

In [47]:
print(X_train.shape)
print(X_test.shape)

(21600, 64, 64, 3)
(5400, 64, 64, 3)


In [48]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')

In [49]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(64,64,3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), input_shape=(64,64,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=2,padding='valid'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 dropout_8 (Dropout)         (None, 64, 64, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               8388864   
                                                      

In [50]:
import random 
buf = list(zip(X_train,Y_train))
random.shuffle(buf)
X_train,Y_train = zip(*buf)
buf = list(zip(X_test,Y_test))
random.shuffle(buf)
X_test,Y_test = zip(*buf)


In [51]:
Y_train=np.asarray(Y_train)
X_test=np.asarray(X_test)
X_train = np.asarray(X_train)
Y_test=np.asarray(Y_test)
print(X_train.shape)
print(X_test.shape)

(21600, 64, 64, 3)
(5400, 64, 64, 3)


In [52]:
results = {
    'AnnualCrop':0,
    'Forest':1,
    'HerbaceousVegetation':2,
    'Highway':3,
    'Industrial':4,
    'Pasture':5,
    'PermanentCrop':6,
    'Residential':7,
    'River':8,
    'SeaLake':9
}
y_train = []
for i in Y_train:
    y_train.append(results.get(i))

In [53]:
y_train = np_utils.to_categorical(y_train)
print(y_train[0])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [54]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(21600, 64, 64, 3)
(21600,)
(5400, 64, 64, 3)
(5400,)


In [55]:
scale = X_train.max()
X_train = X_train/scale
X_test = X_test/scale

mean = X_train.mean()
X_train = X_train - mean
X_test = X_test - mean

In [56]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [57]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1,verbose=1)

Epoch 1/10
608/608 [==============================] - 99s 163ms/step - loss: 1.1419 - categorical_accuracy: 0.6004 - val_loss: 0.7964 - val_categorical_accuracy: 0.7204
Epoch 2/10
608/608 [==============================] - 100s 165ms/step - loss: 0.7374 - categorical_accuracy: 0.7463 - val_loss: 0.7636 - val_categorical_accuracy: 0.7278
Epoch 3/10
608/608 [==============================] - 100s 164ms/step - loss: 0.6097 - categorical_accuracy: 0.7942 - val_loss: 0.5925 - val_categorical_accuracy: 0.7995
Epoch 4/10
608/608 [==============================] - 101s 165ms/step - loss: 0.5389 - categorical_accuracy: 0.8223 - val_loss: 0.5747 - val_categorical_accuracy: 0.8065
Epoch 5/10
608/608 [==============================] - 100s 165ms/step - loss: 0.4877 - categorical_accuracy: 0.8413 - val_loss: 0.6202 - val_categorical_accuracy: 0.8051
Epoch 6/10
608/608 [==============================] - 103s 170ms/step - loss: 0.4585 - categorical_accuracy: 0.8573 - val_loss: 0.7091 - val_categorica